In [1]:
import numpy as np 
from calcbsimpvol import calcbsimpvol
import QuantLib as ql
import math
import numpy as np
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

K=[]
df = pd.read_csv("20200529 2-05pm call.csv")
for i in range(len(df)):
    K.append(df['종목'][i][-5:])

strike=[]
for i in range(len(K)):
    if K.count(K[i])==11:
        strike.append(K[i])

In [2]:
# 1년을 몇일로 볼것인가? 한국은 ACT 즉 실제 그 해의 일수 365로 고정한 것을 씀.
day_count = ql.Actual365Fixed() #Actual/365
calculation_date = ql.Date(29, 5, 2020)

T=[]
S=[]
K=[]
df = pd.read_csv("20200529 2-05pm call.csv")
for i in range(len(df)):
    # print(df['종목'][i][-5:])
    expiry=0
    if df['종목'][i][-5:] in strike:
        if df['종목'][i][-9]=='0':
            if df['종목'][i][-7]=='6':
                expiry = ql.Date(11,6,2020)
            elif df['종목'][i][-7]=='7':
                expiry = ql.Date(9,7,2020)
            elif df['종목'][i][-7]=='8':
                expiry = ql.Date(13,8,2020)
            elif df['종목'][i][-7]=='9':
                expiry = ql.Date(10,9,2020)
            elif df['종목'][i][-8:-6]=='10':
                expiry = ql.Date(8,10,2020)
            elif df['종목'][i][-8:-6]=='11':
                expiry = ql.Date(12,11,2020)
            elif df['종목'][i][-8:-6]=='12':
                expiry = ql.Date(10,12,2020)
        elif df['종목'][i][-9]=='1':
            if df['종목'][i][-7]=='3':
                expiry = ql.Date(11,3,2021)
            elif df['종목'][i][-7]=='6':
                expiry = ql.Date(10,6,2021)
            elif df['종목'][i][-7]=='9':
                expiry = ql.Date(9,9,2021)
            elif df['종목'][i][-8:-6]=='12':
                expiry = ql.Date(9,12,2021)
        elif df['종목'][i][-9]=='2':
            if df['종목'][i][-7]=='3':
                expiry = ql.Date(10,3,2022)
            elif df['종목'][i][-7]=='6':
                expiry = ql.Date(9,6,2022)
            elif df['종목'][i][-7]=='9':
                expiry = ql.Date(8,9,2022)
            elif df['종목'][i][-8:-6]=='12':
                expiry = ql.Date(8,12,2022)

        T.append(round(day_count.yearFraction(calculation_date,expiry),4))
        K.append(float(df['종목'][i][-5:]))
        S.append(float(df['현재가'][i]))

In [3]:
print(T)
print(K)
print(S)

[0.0356, 0.0356, 0.0356, 0.0356, 0.0356, 0.0356, 0.0356, 0.0356, 0.0356, 0.1123, 0.1123, 0.1123, 0.1123, 0.1123, 0.1123, 0.1123, 0.1123, 0.1123, 0.2082, 0.2082, 0.2082, 0.2082, 0.2082, 0.2082, 0.2082, 0.2082, 0.2082, 0.2849, 0.2849, 0.2849, 0.2849, 0.2849, 0.2849, 0.2849, 0.2849, 0.2849, 0.3616, 0.3616, 0.3616, 0.3616, 0.3616, 0.3616, 0.3616, 0.3616, 0.3616, 0.4575, 0.4575, 0.4575, 0.4575, 0.4575, 0.4575, 0.4575, 0.4575, 0.4575, 0.5342, 0.5342, 0.5342, 0.5342, 0.5342, 0.5342, 0.5342, 0.5342, 0.5342, 0.7836, 0.7836, 0.7836, 0.7836, 0.7836, 0.7836, 0.7836, 0.7836, 0.7836, 1.0329, 1.0329, 1.0329, 1.0329, 1.0329, 1.0329, 1.0329, 1.0329, 1.0329, 1.5315, 1.5315, 1.5315, 1.5315, 1.5315, 1.5315, 1.5315, 1.5315, 1.5315, 2.5288, 2.5288, 2.5288, 2.5288, 2.5288, 2.5288, 2.5288, 2.5288, 2.5288]
[220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 220.0, 230.0, 240

In [4]:
implied_volatility=[]
for i in range(len(T)):
    s = np.asarray(267.82) # Underlying Price : Kospi200지수 5.29 2:05pm
    k = np.asarray([K[i]]) # Strike Price 
    tau = np.asarray([T[i]]) # Time to Maturity 
    r = np.asarray(0.81)  # CD91일물 금리 2020.5.21일 기준
    q = np.asarray(0.00) # Dividend Rate 
    cp = np.asarray(-1) # Option Type 
    P = np.asarray([S[i]]) # Market Price
    
    imp_vol = calcbsimpvol(dict(cp=cp, P=P, S=s, K=k, tau=tau, r=r, q=q))
    if imp_vol[0][0]>=1:
        imp_vol[0][0]=nan
     
    implied_volatility.append(imp_vol[0][0])
print(implied_volatility)

[nan, nan, nan, nan, 0.8860942186534665, 0.33688611390160944, 0.006238301729020898, nan, nan, nan, nan, nan, nan, 0.7791568462483932, 0.4762093675042754, 0.24835053064409712, 0.10405855066959094, 0.009944871079340856, nan, nan, nan, nan, 0.7985196116101432, 0.6055726136530943, 0.3902972600486157, 0.2400629877855558, 0.1650832883236796, nan, nan, nan, nan, 0.8652370719738937, 0.678926298852141, 0.5000190913255863, 0.3527878524475413, 0.25445948827931036, nan, nan, nan, nan, 0.9239030257007631, 0.7261688465416963, 0.5612152721219652, 0.43032085302616196, 0.32826959180725535, nan, nan, nan, nan, 0.9627557991393693, 0.7842785057247864, 0.6358546160938803, 0.5144297319087867, 0.4163939295876253, nan, nan, nan, nan, nan, 0.8690118031164263, 0.7302137760391457, 0.5957099897808181, 0.4829251552849719, nan, nan, nan, nan, nan, 0.9547758984207024, 0.8188016950266089, 0.707872015838089, 0.6129156724299214, nan, nan, nan, nan, nan, nan, 0.9829944880373872, 0.8676076294885422, 0.7646908619007262, n

In [5]:
K=list(set(K))
T=list(set(T))
print(K)
print(T)

[290.0, 260.0, 230.0, 300.0, 270.0, 240.0, 280.0, 250.0, 220.0]
[0.2849, 0.3616, 0.1123, 0.2082, 0.4575, 0.5342, 0.7836, 1.0329, 1.5315, 2.5288, 0.0356]


In [6]:
K.sort()
T.sort()
print(K)
print(T)


[220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0]
[0.0356, 0.1123, 0.2082, 0.2849, 0.3616, 0.4575, 0.5342, 0.7836, 1.0329, 1.5315, 2.5288]


In [40]:
TM=np.asarray(T)
KP=np.asarray(K)
TM,KP=np.meshgrid(TM,KP)
imp_vol=np.asarray(implied_volatility)
imp_vol=np.reshape(imp_vol,(len(T),len(K))).T
print(imp_vol)

[[       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan]
 [       nan        nan        nan        nan        nan        nan
         nan        nan        nan        nan        nan]
 [0.8289963  0.7441218  0.77397965 0.84482072 0.90621969 0.94753386
         nan        nan        nan        nan        nan]
 [0.25437994 0.43938909 0.580534   0.65837261 0.70859102 0.76919041
  0.85518749 0.94409239        nan        nan        nan]
 [       nan 0.21183587 0.36639617 0.47999757 0.54410338 0.62109992
  0.71656846 0.80834191 0.97408669        nan        nan]
 [       nan 0.06751006 0.21821991 0.33393533 0.41394121 0.50016428
  0.58249879 0.69767395 0.85888502        nan     

In [23]:

trace = go.Surface(x=TM, y=KP, z=imp_vol, colorscale='Jet', opacity=0.8)
data = [trace] 
layout = go.Layout(title='Call Theta Surface', 
        scene={'xaxis':{'title':'Maturity'}, 'yaxis':{'title':'Spot Price'}, 'zaxis':{'title':'Theta'}}, 
        width=800, height=800, autosize=False, margin=dict(pad=0) ) 
fig = go.Figure(data=data, layout=layout) 
iplot(fig)

In [41]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace = go.Surface(x=TM, y=KP, z=imp_vol, colorscale='Jet', opacity=0.8,contours_x=dict(show=True, color="black", start=0, end=1, size=(1 - 0)/50, project_x=True), contours_y=dict(show=True, color="black", start=200, end=300, size=(300 - 200)/50, project_y=True))
data = [trace] 
layout = go.Layout(title='Implied Volatility Surface of call option', 
        scene={'xaxis':{'title':'Maturity'}, 'yaxis':{'title':'Strike Price'}, 'zaxis':{'title':'implied volatility'}}, 
        width=1000, height=1000, autosize=False, margin=dict(pad=0) ) 
fig = go.Figure(data=data, layout=layout) 
iplot(fig)

In [48]:
K=[]
df = pd.read_csv("20200529 2-05pm put.csv")
for i in range(len(df)):
    K.append(df['종목'][i][-5:])

strike=[]
for i in range(len(K)):
    if K.count(K[i])==11:
        strike.append(K[i])

In [49]:
# 1년을 몇일로 볼것인가? 한국은 ACT 즉 실제 그 해의 일수 365로 고정한 것을 씀.
day_count = ql.Actual365Fixed() #Actual/365
calculation_date = ql.Date(29, 5, 2020)

T=[]
S=[]
K=[]
df = pd.read_csv("20200529 2-05pm put.csv")
for i in range(len(df)):
    # print(df['종목'][i][-5:])
    expiry=0
    if df['종목'][i][-5:] in strike:
        if df['종목'][i][-9]=='0':
            if df['종목'][i][-7]=='6':
                expiry = ql.Date(11,6,2020)
            elif df['종목'][i][-7]=='7':
                expiry = ql.Date(9,7,2020)
            elif df['종목'][i][-7]=='8':
                expiry = ql.Date(13,8,2020)
            elif df['종목'][i][-7]=='9':
                expiry = ql.Date(10,9,2020)
            elif df['종목'][i][-8:-6]=='10':
                expiry = ql.Date(8,10,2020)
            elif df['종목'][i][-8:-6]=='11':
                expiry = ql.Date(12,11,2020)
            elif df['종목'][i][-8:-6]=='12':
                expiry = ql.Date(10,12,2020)
        elif df['종목'][i][-9]=='1':
            if df['종목'][i][-7]=='3':
                expiry = ql.Date(11,3,2021)
            elif df['종목'][i][-7]=='6':
                expiry = ql.Date(10,6,2021)
            elif df['종목'][i][-7]=='9':
                expiry = ql.Date(9,9,2021)
            elif df['종목'][i][-8:-6]=='12':
                expiry = ql.Date(9,12,2021)
        elif df['종목'][i][-9]=='2':
            if df['종목'][i][-7]=='3':
                expiry = ql.Date(10,3,2022)
            elif df['종목'][i][-7]=='6':
                expiry = ql.Date(9,6,2022)
            elif df['종목'][i][-7]=='9':
                expiry = ql.Date(8,9,2022)
            elif df['종목'][i][-8:-6]=='12':
                expiry = ql.Date(8,12,2022)

        T.append(round(day_count.yearFraction(calculation_date,expiry),4))
        K.append(float(df['종목'][i][-5:]))
        S.append(float(df['현재가'][i]))

In [50]:
print(T)
print(K)
print(S)

[0.0329, 0.0329, 0.0329, 0.0329, 0.0329, 0.0329, 0.0329, 0.0329, 0.0329, 0.1096, 0.1096, 0.1096, 0.1096, 0.1096, 0.1096, 0.1096, 0.1096, 0.1096, 0.2055, 0.2055, 0.2055, 0.2055, 0.2055, 0.2055, 0.2055, 0.2055, 0.2055, 0.2822, 0.2822, 0.2822, 0.2822, 0.2822, 0.2822, 0.2822, 0.2822, 0.2822, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.3589, 0.4548, 0.4548, 0.4548, 0.4548, 0.4548, 0.4548, 0.4548, 0.4548, 0.4548, 0.5315, 0.5315, 0.5315, 0.5315, 0.5315, 0.5315, 0.5315, 0.5315, 0.5315, 0.7808, 0.7808, 0.7808, 0.7808, 0.7808, 0.7808, 0.7808, 0.7808, 0.7808, 1.0301, 1.0301, 1.0301, 1.0301, 1.0301, 1.0301, 1.0301, 1.0301, 1.0301, 1.5288, 1.5288, 1.5288, 1.5288, 1.5288, 1.5288, 1.5288, 1.5288, 1.5288, 2.526, 2.526, 2.526, 2.526, 2.526, 2.526, 2.526, 2.526, 2.526]
[220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0, 220.0, 230.0, 240.0, 250.0

In [51]:
implied_volatility=[]
for i in range(len(T)):
    s = np.asarray(267.82) # Underlying Price : Kospi200 5.29
    k = np.asarray([K[i]]) # Strike Price 
    tau = np.asarray([T[i]]) # Time to Maturity 
    r = np.asarray(0.81)  # CD91일물 금리 2020.5.29일 기준
    q = np.asarray(0.00) # Dividend Rate 
    cp = np.asarray(-1) # Option Type 
    P = np.asarray([S[i]]) # Market Price
    
    imp_vol = calcbsimpvol(dict(cp=cp, P=P, S=s, K=k, tau=tau, r=r, q=q))
    if imp_vol[0][0]>=1:
        imp_vol[0][0]=nan
     
    implied_volatility.append(imp_vol[0][0])
print(implied_volatility)

[0.5710676642627747, 0.5109519756242066, 0.4543216032753612, 0.403639011157178, 0.3710253935602258, 0.3801906828809589, 0.47954929730960966, 0.4639719969392601, 0.7925252309453259, 0.5241654409547133, 0.5028918867321466, 0.49031599892972144, 0.4863930570509216, 0.5021004736824913, 0.5809459341451797, 0.6046256157867563, 0.7136734219430239, 0.9283216868036579, 0.5759885426692134, 0.5823593171697453, 0.5807603896354029, 0.6106419142787474, 0.6687173288725021, 0.7355512103678566, 0.8125821795462381, 0.8978496953354327, 0.9902402467968705, 0.6114539051591592, 0.6351839655051917, 0.6066788379005753, 0.6605864882500453, 0.7200220008306222, 0.8004577417097373, 0.8875749531353891, 0.9769149940827448, nan, 0.7089465974048538, 0.7545096944647823, 0.806130646902108, 0.8597450357426318, 0.9151075817472735, 0.9838093014102788, nan, nan, nan, 0.7477114503056528, 0.7733296172898941, 0.8154412706614302, 0.8592004280733923, 0.9021462089208143, 0.9646120543326735, nan, nan, nan, 0.7793310151551156, 0.76

In [52]:
K=list(set(K))
T=list(set(T))
print(K)
print(T)

[290.0, 260.0, 230.0, 300.0, 270.0, 240.0, 280.0, 250.0, 220.0]
[0.0329, 0.2822, 0.2055, 0.3589, 0.1096, 0.4548, 0.5315, 0.7808, 1.0301, 1.5288, 2.526]


In [53]:
K.sort()
T.sort()
print(K)
print(T)

[220.0, 230.0, 240.0, 250.0, 260.0, 270.0, 280.0, 290.0, 300.0]
[0.0329, 0.1096, 0.2055, 0.2822, 0.3589, 0.4548, 0.5315, 0.7808, 1.0301, 1.5288, 2.526]


In [54]:
TM=np.asarray(T)
KP=np.asarray(K)
TM,KP=np.meshgrid(TM,KP)
imp_vol=np.asarray(implied_volatility)
imp_vol=np.reshape(imp_vol,(len(T),len(K))).T
print(imp_vol)

[[0.57106766 0.52416544 0.57598854 0.61145391 0.7089466  0.74771145
  0.77933102 0.91092781        nan        nan        nan]
 [0.51095198 0.50289189 0.58235932 0.63518397 0.75450969 0.77332962
  0.76252748        nan        nan        nan        nan]
 [0.4543216  0.490316   0.58076039 0.60667884 0.80613065 0.81544127
  0.79324375        nan        nan        nan        nan]
 [0.40363901 0.48639306 0.61064191 0.66058649 0.85974504 0.85920043
  0.85124271        nan        nan        nan        nan]
 [0.37102539 0.50210047 0.66871733 0.720022   0.91510758 0.90214621
  0.89997253        nan        nan        nan        nan]
 [0.38019068 0.58094593 0.73555121 0.80045774 0.9838093  0.96461205
  0.92867874        nan        nan        nan        nan]
 [0.4795493  0.60462562 0.81258218 0.88757495        nan        nan
         nan        nan        nan        nan        nan]
 [0.463972   0.71367342 0.8978497  0.97691499        nan        nan
         nan        nan        nan        nan     

In [55]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

trace = go.Surface(x=TM, y=KP, z=imp_vol, colorscale='Jet', opacity=0.8,contours_x=dict(show=True, color="black", start=0, end=1, size=(1 - 0)/50, project_x=True), contours_y=dict(show=True, color="black", start=200, end=300, size=(300 - 200)/50, project_y=True))
data = [trace] 
layout = go.Layout(title='Implied Volatility Surface of put option', 
        scene={'xaxis':{'title':'Maturity'}, 'yaxis':{'title':'Strike Price'}, 'zaxis':{'title':'implied volatility'}}, 
        width=1000, height=1000, autosize=False, margin=dict(pad=0) ) 
fig = go.Figure(data=data, layout=layout) 
iplot(fig)